Houses dataset

In [1]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 31.64 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (2144/2144), done.


In [21]:
# where is HousesInfo.txt ???
import os

for root, dirs, files in os.walk("/content/Houses-dataset/Houses Dataset"):
  for f in files:
    if f.split('.')[-1] == 'txt':
      print(os.path.join(root, f))


/content/Houses-dataset/Houses Dataset/HousesInfo.txt


In [2]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os


def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

	# determine (1) the unique zip codes and (2) the number of data points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()

	# loop over each of the unique zip codes and their corresponding count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)

	# return the data frame
	return df

In [3]:
inputPath = "/content/Houses-dataset/Houses Dataset/HousesInfo.txt"

load_house_attributes(inputPath)


,bedrooms,bathrooms,area,zipcode,price
30,5,3.0,2520,93446,789000
32,3,2.0,1802,93446,365000
39,3,3.0,2146,93446,455000
80,4,2.5,2464,91901,599000
81,2,2.0,1845,91901,529800
...,...,...,...,...,...
499,4,4.0,3000,93446,1495000
500,3,2.0,2330,93446,599900
501,3,2.5,1339,93446,344900
502,3,2.0,1472,93446,309995


In [ ]:
def process_house_attributes(df, train, test):
  # initialize the column names of the continuous data
  continuous = ["bedrooms", "bathrooms", "area"]

  # perform min-max scaling each continuous feature column to the range [0, 1]
  cs = MinMaxScaler()
  trainContinuous = cs.fit_transform(train[continuous])
  testContinuous = cs.transform(test[continuous])

  # one-hot encode the zip code categorical data (by definition of
  # one-hot encoing, all output features are now in the range [0, 1])
  zipBinarizer = LabelBinarizer().fit(df["zipcode"])
  trainCategorical = zipBinarizer.transform(train["zipcode"])
  testCategorical = zipBinarizer.transform(test["zipcode"])

  # construct our training and testing data points by concatenating
  # the categorical features with the continuous features
  trainX = np.hstack([trainCategorical, trainContinuous])
  testX = np.hstack([testCategorical, testContinuous])

  # return the concatenated training and testing data
  return (trainX, testX)

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_mlp(dim, regress=False):
  # define our MLP network
  model = Sequential()
  model.add(Dense(512, input_dim=dim, activation="relu"))
  model.add(Dense(256, activation="relu"))
  model.add(Dense(64, activation="relu"))
  model.add(Dense(4, activation="relu"))

  # check to see if the regression node should be added
  if regress:
    model.add(Dense(1, activation="linear"))
  # return our model
  return model


In [ ]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os

inputPath = "/content/Houses-dataset/Houses Dataset/HousesInfo.txt"

df = load_house_attributes(inputPath)
# construct a training and testing split with 75% of the data used
# for training and the remaining 25% for evaluation
(train, test) = train_test_split(df, test_size=0.25, random_state=42)

maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
# [INFO] processing data
(trainX, testX) = process_house_attributes(df, train, test)


# create our MLP and then compile the model using mean absolute
# percentage error as our loss, implying that we seek to minimize
# the absolute percentage difference between our price *predictions*
# and the *actual prices*
model = create_mlp(trainX.shape[1], regress=True)

model.compile(loss="mean_absolute_percentage_error", optimizer='Adam')

# train the model
print("[INFO] training model...")
model.fit(x=trainX, y=trainY,
	validation_data=(testX, testY),
	epochs=200, batch_size=8)

[INFO] training model...
Epoch 1/200
34/34 [==============================] - 2s 14ms/step - loss: 50.0521 - val_loss: 44.7075
Epoch 2/200
34/34 [==============================] - 0s 10ms/step - loss: 36.5598 - val_loss: 30.3653
Epoch 3/200
34/34 [==============================] - 0s 10ms/step - loss: 29.8336 - val_loss: 27.5729
Epoch 4/200
34/34 [==============================] - 0s 9ms/step - loss: 28.0566 - val_loss: 26.8745
Epoch 5/200
34/34 [==============================] - 0s 11ms/step - loss: 26.7702 - val_loss: 21.7125
Epoch 6/200
34/34 [==============================] - 0s 11ms/step - loss: 26.5367 - val_loss: 25.6252
Epoch 7/200
34/34 [==============================] - 0s 11ms/step - loss: 25.3498 - val_loss: 30.1466
Epoch 8/200
34/34 [==============================] - 0s 11ms/step - loss: 27.1091 - val_loss: 34.1324
Epoch 9/200
34/34 [==============================] - 0s 7ms/step - loss: 30.1158 - val_loss: 25.3826
Epoch 10/200
34/34 [==============================] - 0s 6m

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testX)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY     # flatten()  ==> reshape(91,)
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
3/3 [==============================] - 0s 5ms/step
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 21.34%, std: 19.36%


## **keras-regression-and-cnns**

In [4]:
inputPath_df = "/content/Houses-dataset/Houses Dataset/HousesInfo.txt"
df = load_house_attributes(inputPath= inputPath_df)
df

,bedrooms,bathrooms,area,zipcode,price
30,5,3.0,2520,93446,789000
32,3,2.0,1802,93446,365000
39,3,3.0,2146,93446,455000
80,4,2.5,2464,91901,599000
81,2,2.0,1845,91901,529800
...,...,...,...,...,...
499,4,4.0,3000,93446,1495000
500,3,2.0,2330,93446,599900
501,3,2.5,1339,93446,344900
502,3,2.0,1472,93446,309995


In [5]:
inputPath = "/content/Houses-dataset/Houses Dataset"

def load_house_images(df, inputPath):
	# initialize our images array (i.e., the house images themselves)
	images = []

	# loop over the indexes of the houses
	for i in df.index.values:
		# find the four images for the house and sort the file paths,
		# ensuring the four are always in the *same order*
		basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
		housePaths = sorted(list(glob.glob(basePath)))

    # initialize our list of input images along with the output image
		# after *combining* the four input images
		inputImages = []
		outputImage = np.zeros((64, 64, 3), dtype="uint8")

		# loop over the input house paths
		for housePath in housePaths:
			# load the input image, resize it to be 32 32, and then
			# update the list of input images
			image = cv2.imread(housePath)
			image = cv2.resize(image, (32, 32))
			inputImages.append(image)

    # tile the four input images in the output image such the first
		# image goes in the top-right corner, the second image in the
		# top-left corner, the third image in the bottom-right corner,
		# and the final image in the bottom-left corner
		outputImage[0:32, 0:32] = inputImages[0]
		outputImage[0:32, 32:64] = inputImages[1]
		outputImage[32:64, 32:64] = inputImages[2]
		outputImage[32:64, 0:32] = inputImages[3]



    # another way to attach images:
    # up = np.hstack([inputImages[0],inputImages[1]])
    # down = np.hstack([inputImages[3],inputImages[2]])
    # img = np.vstack([up, down])
    # img.shape



		# add the tiled image to our set of images the network will be
		# trained on
		images.append(outputImage)

	# return our set of images
	return np.array(images)

In [6]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
  # initialize the input shape and channel dimension, assuming
  # TensorFlow/channels-last ordering
  inputShape = (height, width, depth)
  chanDim = -1

  # define the model input
  inputs = Input(shape=inputShape, batch_size=8)

  # loop over the number of filters
  for (i, f) in enumerate(filters):

    # if this is the first CONV layer then set the input appropriately
    if i == 0:
      x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

  # flatten the volume, then FC => RELU => BN => DROPOUT
  x = Flatten()(x)
  x = Dense(16)(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Dropout(0.5)(x)

  # apply another FC layer, this one to match the number of nodes
  # coming out of the MLP
  x = Dense(4)(x)
  x = Activation("relu")(x)

  # check to see if the regression node should be added
  if regress:
    x = Dense(1, activation="linear")(x)

  # construct the CNN
  model = Model(inputs, x)

  # return the CNN
  return model

In [7]:
# import the necessary packages
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os

# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
inputPath_df = "/content/Houses-dataset/Houses Dataset/HousesInfo.txt"
df = load_house_attributes(inputPath= inputPath_df)

# load the house images and then scale the pixel intensities to the
# range [0, 1]
print("[INFO] loading house images...")
inputPath = "/content/Houses-dataset/Houses Dataset"
images = load_house_images(df, inputPath)
images = images / 255.0

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainAttrX, testAttrX, trainImagesX, testImagesX) = train_test_split(df, images, test_size=0.2, random_state=42)



# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# price *predictions* and the *actual prices*
model = create_cnn(64, 64, 3, regress=True)
model.compile(loss="mean_absolute_percentage_error", optimizer="Adam")

# train the model
print("[INFO] training model...")
#callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss')

model.fit(x=trainImagesX, y=trainY,
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)


[INFO] loading house attributes...
[INFO] loading house images...
[INFO] training model...
Epoch 1/200
37/37 [==============================] - 13s 18ms/step - loss: 1963.3849 - val_loss: 2048.5405
Epoch 2/200
37/37 [==============================] - 0s 7ms/step - loss: 1571.0656 - val_loss: 4780.1367
Epoch 3/200
37/37 [==============================] - 0s 7ms/step - loss: 1035.9656 - val_loss: 5742.0044
Epoch 4/200
37/37 [==============================] - 0s 8ms/step - loss: 671.7953 - val_loss: 7352.5327
Epoch 5/200
37/37 [==============================] - 0s 7ms/step - loss: 504.6903 - val_loss: 9308.5938
Epoch 6/200
37/37 [==============================] - 0s 6ms/step - loss: 427.6508 - val_loss: 9661.9346
Epoch 7/200
37/37 [==============================] - 0s 7ms/step - loss: 269.7313 - val_loss: 8326.3643
Epoch 8/200
37/37 [==============================] - 0s 8ms/step - loss: 329.0517 - val_loss: 6586.5361
Epoch 9/200
37/37 [==============================] - 0s 8ms/step - loss:

In [8]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testImagesX)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))


[INFO] predicting house prices...
3/3 [==============================] - 0s 43ms/step
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 86.26%, std: 103.72%


## **Keras: Multiple Inputs and Mixed Data**

https://pyimagesearch.com/wp-content/uploads/2019/02/keras_multi_input_header.png

https://pyimagesearch.com/wp-content/uploads/2019/02/keras_multi_input_mixed_data.png

https://pyimagesearch.com/wp-content/uploads/2019/02/keras_multi_input_arch.png

In [1]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (2144/2144), done.


In [2]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os


def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

	# determine (1) the unique zip codes and (2) the number of data points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()

	# loop over each of the unique zip codes and their corresponding count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)

	# return the data frame
	return df

In [3]:
inputPath = "/content/Houses-dataset/Houses Dataset/HousesInfo.txt"

df = pd.read_csv(inputPath, sep = " ", names=["bedrooms", "bathrooms", "area", "zipcode", "price"] )
df

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226
...,...,...,...,...,...
530,5,2.0,2066,94531,399900
531,4,3.5,9536,94531,460000
532,3,2.0,2014,94531,407000
533,4,3.0,2312,94531,419000


In [4]:
def process_house_attributes(df, train, test):
  # initialize the column names of the continuous data
  continuous = ["bedrooms", "bathrooms", "area"]

  # perform min-max scaling each continuous feature column to the range [0, 1]
  cs = MinMaxScaler()
  trainContinuous = cs.fit_transform(train[continuous])
  testContinuous = cs.transform(test[continuous])

  # one-hot encode the zip code categorical data (by definition of
  # one-hot encoing, all output features are now in the range [0, 1])
  zipBinarizer = LabelBinarizer().fit(df["zipcode"])
  trainCategorical = zipBinarizer.transform(train["zipcode"])
  testCategorical = zipBinarizer.transform(test["zipcode"])

  # construct our training and testing data points by concatenating
  # the categorical features with the continuous features
  trainX = np.hstack([trainCategorical, trainContinuous])
  testX = np.hstack([testCategorical, testContinuous])

  # return the concatenated training and testing data
  return (trainX, testX)

In [5]:
inputPath = "/content/Houses-dataset/Houses Dataset"

def load_house_images(df, inputPath):
	# initialize our images array (i.e., the house images themselves)
	images = []

	# loop over the indexes of the houses
	for i in df.index.values:
		# find the four images for the house and sort the file paths,
		# ensuring the four are always in the *same order*
		basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
		housePaths = sorted(list(glob.glob(basePath)))

    # initialize our list of input images along with the output image
		# after *combining* the four input images
		inputImages = []
		outputImage = np.zeros((64, 64, 3), dtype="uint8")

		# loop over the input house paths
		for housePath in housePaths:
			# load the input image, resize it to be 32 32, and then
			# update the list of input images
			image = cv2.imread(housePath)
			image = cv2.resize(image, (32, 32))
			inputImages.append(image)

    # tile the four input images in the output image such the first
		# image goes in the top-right corner, the second image in the
		# top-left corner, the third image in the bottom-right corner,
		# and the final image in the bottom-left corner
		outputImage[0:32, 0:32] = inputImages[0]
		outputImage[0:32, 32:64] = inputImages[1]
		outputImage[32:64, 32:64] = inputImages[2]
		outputImage[32:64, 0:32] = inputImages[3]



    # another way to attach images:
    # up = np.hstack([inputImages[0],inputImages[1]])
    # down = np.hstack([inputImages[3],inputImages[2]])
    # img = np.vstack([up, down])
    # img.shape



		# add the tiled image to our set of images the network will be
		# trained on
		images.append(outputImage)

	# return our set of images
	return np.array(images)

In [6]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_mlp(dim, regress=False):
  # define our MLP network
  model = Sequential()
  model.add(Dense(8, input_dim=dim, activation="relu"))
  model.add(Dense(4, activation="relu"))


  # check to see if the regression node should be added
  if regress:
    model.add(Dense(1, activation="linear"))
  # return our model
  return model


In [7]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
  # initialize the input shape and channel dimension, assuming
  # TensorFlow/channels-last ordering
  inputShape = (height, width, depth)
  chanDim = -1

  # define the model input
  inputs = Input(shape=inputShape, batch_size=8)

  # loop over the number of filters
  for (i, f) in enumerate(filters):

    # if this is the first CONV layer then set the input appropriately
    if i == 0:
      x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

  # flatten the volume, then FC => RELU => BN => DROPOUT
  x = Flatten()(x)
  x = Dense(16)(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Dropout(0.5)(x)

  # apply another FC layer, this one to match the number of nodes
  # coming out of the MLP
  x = Dense(4)(x)
  x = Activation("relu")(x)

  # check to see if the regression node should be added
  if regress:
    x = Dense(1, activation="linear")(x)

  # construct the CNN
  model = Model(inputs, x)

  # return the CNN
  return model

**Multiple inputs with Keras**

In [9]:
# import the necessary packages
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
import numpy as np
import argparse
import locale
import os



print("[INFO] loading house attributes...")
inputPath = "/content/Houses-dataset/Houses Dataset/HousesInfo.txt"
df = load_house_attributes(inputPath)

print("[INFO] loading house images...")
inputPath_images = "/content/Houses-dataset/Houses Dataset"
images = load_house_images(df, inputPath_images)
images = images / 255.0


# partition the data into training and testing splits using 75% of the data for training and the remaining 25% for testing
(trainAttrX, testAttrX, trainImagesX, testImagesX) = train_test_split(df, images, test_size=0.25, random_state=42)



# convert label range between [0, 1]
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice



# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
(trainAttrX, testAttrX) = process_house_attributes(df, trainAttrX, testAttrX)



# create the MLP and CNN models
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(64, 64, 3, regress=False)

# create the input to our final set of layers as the *output* of both the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)



model.compile(loss="mean_absolute_percentage_error", optimizer='Adam')

# train the model
model.fit(
	x=[trainAttrX, trainImagesX], y=trainY,
	validation_data=([testAttrX, testImagesX], testY),
	epochs=200, batch_size=8)


[INFO] loading house attributes...
[INFO] loading house images...
Epoch 1/200
34/34 [==============================] - 15s 36ms/step - loss: 1432.6733 - val_loss: 206.7444
Epoch 2/200
34/34 [==============================] - 0s 11ms/step - loss: 662.3272 - val_loss: 312.0533
Epoch 3/200
34/34 [==============================] - 0s 8ms/step - loss: 392.1815 - val_loss: 978.6492
Epoch 4/200
34/34 [==============================] - 0s 9ms/step - loss: 318.6018 - val_loss: 926.0029
Epoch 5/200
34/34 [==============================] - 0s 12ms/step - loss: 256.9643 - val_loss: 1121.7869
Epoch 6/200
34/34 [==============================] - 0s 12ms/step - loss: 196.3510 - val_loss: 1259.2904
Epoch 7/200
34/34 [==============================] - 0s 14ms/step - loss: 168.5683 - val_loss: 1244.3789
Epoch 8/200
34/34 [==============================] - 0s 13ms/step - loss: 162.0541 - val_loss: 646.7910
Epoch 9/200
34/34 [==============================] - 0s 13ms/step - loss: 169.9431 - val_loss: 498.

In [10]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict([testAttrX, testImagesX])

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
3/3 [==============================] - 1s 88ms/step
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 23.12%, std: 19.60%
